In [1]:
## imports
import importlib
import osmosys.osmosys
import osmosys.Backups
import activityinfo
import pandas as pd
import model.modelAI
from IPython.display import display
import json
import os.path
from urllib.error import HTTPError

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo)

<module 'activityinfo' from 'D:\\proyectos\\OSMOSYS_2023\\IntegracionAI2023\\AI_OsmosysIntegrationScripts2023\\activityinfo\\__init__.py'>

In [2]:
month = 'ENERO'
osmosysData = osmosys.osmosys.get_ai_form_level_1()
print(osmosysData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   folder              260 non-null    object
 1   form                260 non-null    object
 2   form_id             260 non-null    object
 3   parent_form         260 non-null    object
 4   parent_form_id      260 non-null    object
 5   org_acron           260 non-null    object
 6   id_registro_imp     259 non-null    object
 7   osmosys_code        260 non-null    object
 8   canton              260 non-null    object
 9   canton_id_registro  260 non-null    object
dtypes: object(10)
memory usage: 20.4+ KB
None


In [5]:
# get all unique form ids
#osmosysData.form_id.value_counts()
parentFormsIds = osmosysData.parent_form_id.unique()
print(parentFormsIds)
print(type(parentFormsIds))
print(osmosysData.groupby(['parent_form', 'parent_form_id']).size())

['ch4yhkpldnik44oigh' 'cpfsjn6ldnik44oig2' 'cg7ke70ldnik44oih1'
 'c235tqcldnik44oigt' 'crphcl4ldnik44oig8' 'cbky5k2ldnik44oige'
 'cmmtdkuldnik44oigw' 'cab54y8ldnik44oih3' 'cgdo4q4ldnik44oig6']
<class 'numpy.ndarray'>
parent_form                               parent_form_id    
Alojamiento                               cgdo4q4ldnik44oig6    21
Comunicación                              ch4yhkpldnik44oigh     5
Educación                                 cpfsjn6ldnik44oig2    35
Integración                               cg7ke70ldnik44oih1    48
Protección VBG                            cmmtdkuldnik44oigw    30
Protección a la infancia                  crphcl4ldnik44oig8    31
Protección general                        cbky5k2ldnik44oige    68
Salud                                     cab54y8ldnik44oih3    20
Transferencias monetarias multipropósito  c235tqcldnik44oigt     2
dtype: int64


In [6]:
importlib.reload(osmosys.osmosys)
aiToken = osmosys.osmosys.getToken()
formTotalDf = None
for parentFormsId in parentFormsIds:
    print(parentFormsId)
    client = activityinfo.Client(token=aiToken, base_url='https://www.activityinfo.org/resources')
    try:
        dbsJson = client.get_resource('form/{parentFormsId}/query'.format(parentFormsId=parentFormsId))
        dbsDf = pd.DataFrame.from_records(dbsJson)
        dbsDf['parent_form_id'] = parentFormsId
        print(dbsDf.shape)
        if formTotalDf is None:
            formTotalDf = dbsDf
        else:
            formTotalDf = pd.concat([formTotalDf, dbsDf], axis=0)
    except Exception:
        print("Oops!  No existe el registro")
        print(Exception)
    print(formTotalDf.shape)

ch4yhkpldnik44oigh
(6, 33)
(6, 33)
cpfsjn6ldnik44oig2
(36, 33)
(42, 34)
cg7ke70ldnik44oih1
(49, 33)
(91, 35)
c235tqcldnik44oigt
(6, 33)
(97, 36)
crphcl4ldnik44oig8
(32, 33)
(129, 37)
cbky5k2ldnik44oige
(68, 33)
(197, 38)
cmmtdkuldnik44oigw
(31, 33)
(228, 39)
cab54y8ldnik44oih3
(35, 33)
(263, 40)
cgdo4q4ldnik44oig6
(27, 33)
(290, 41)


In [51]:
# ordeno para quitar duplicados
# filtro implementaciòn indirecta y doante acnur
print('formTotalDf: ' + str(formTotalDf.shape))
formPartnersDf = formTotalDf.loc[
    (formTotalDf['implementacion'] == 'Indirecta') & (formTotalDf['donante.@id'] == 'chgxs46l02wy5i81e')]
print('formPartnersDf: ' + str(formPartnersDf.shape))

formTotalDf: (2, 34)
formPartnersDf: (1, 34)


In [52]:
formPartnersDfClean = formPartnersDf.sort_values('@lastEditTime', ascending=False).drop_duplicates(
    subset=['implementador.@id', 'ubicacion.@id', 'parent_form_id'])
print('formPartnersDfClean: ' + str(formPartnersDfClean.shape))
formPartnersDfClean = formPartnersDfClean[
    ["@id", "@lastEditTime", "implementacion", "donante.@id", "donante.org_nombre", "donante.donante",
     "implementador.@id",
     "implementador.org_nombre", "implementador.donante", "ubicacion.@id", "ubicacion.name", "ubicacion.code",
     "ubicacion.parent.name", "ubicacion.parent.code", "codigo_referencia", "parent_form_id"
     ]]

formPartnersDfClean: (1, 34)


In [53]:
## hago merge para buscar cuales falta por crear
mergeDf = pd.merge(osmosysData, formPartnersDfClean, how='left',
                   left_on=['parent_form_id', 'id_registro_imp', 'canton_id_registro'],
                   right_on=['parent_form_id', 'implementador.@id', 'ubicacion.@id'])
formsToCreate = mergeDf[mergeDf['@id'].isna()]
numberOfFormTocreate = formsToCreate.shape[0]
print('Forms to create: ' + str(numberOfFormTocreate))

Forms to create: 273


In [57]:
importlib.reload(model.modelAI)

## creo las estructuras de datos
changesList = []
newIds = []
for index, row in formsToCreate.iterrows():
    form = model.modelAI.Form(cqes12jkykah3la9='40038', org_user='cc9fjcdkyyk18oj6', implementacion='c89eykqkykc8oig1b',
                              donante='chgxs46l02wy5i81e',
                              implementador=row['id_registro_imp'],
                              ubicacion=row['canton_id_registro'])
    newId = activityinfo.generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(row['parent_form_id'], newId, None, form)
    changesList.append(record)

changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)

In [59]:
osmosys.Backups.do_backup('form1_canton_org','',month,2023, changesList, finalJson)

<class 'list'>
 creado respaldo: 2023-ENERO\data_form1_canton_org-.json


In [60]:
## send to AI
try:
    if (numberOfFormTocreate > 0):
        client.post_resource('update', body=finalJson)
        print("Enviado a AI")
    else:
        print("nada por crear en AI")
except HTTPError as e:
    code = e.response.status_code
    print('error:')
    print(code)
    print(e)

Enviado a AI
